In [1]:
import math
from dataclasses import dataclass
from typing import Union

import torch
import torch.nn as nn
import torch.nn.functional as F



class PScan(torch.autograd.Function):
    @staticmethod
    def pscan(A, X):
        # A : (B, D, L, N)
        # X : (B, D, L, N)

        # modifies X in place by doing a parallel scan.
        # more formally, X will be populated by these values :
        # H[t] = A[t] * H[t-1] + X[t] with H[0] = 0
        # which are computed in parallel (2*log2(T) sequential steps (ideally), instead of T sequential steps)

        # only supports L that is a power of two (mainly for a clearer code)
        
        B, D, L, _ = A.size()
        num_steps = int(math.log2(L))

        # up sweep (last 2 steps unfolded)
        Aa = A
        Xa = X
        for _ in range(num_steps-2):
            T = Xa.size(2)
            Aa = Aa.view(B, D, T//2, 2, -1)
            Xa = Xa.view(B, D, T//2, 2, -1)
            
            Xa[:, :, :, 1].add_(Aa[:, :, :, 1].mul(Xa[:, :, :, 0]))
            Aa[:, :, :, 1].mul_(Aa[:, :, :, 0])

            Aa = Aa[:, :, :, 1]
            Xa = Xa[:, :, :, 1]

        # we have only 4, 2 or 1 nodes left
        if Xa.size(2) == 4:
            Xa[:, :, 1].add_(Aa[:, :, 1].mul(Xa[:, :, 0]))
            Aa[:, :, 1].mul_(Aa[:, :, 0])

            Xa[:, :, 3].add_(Aa[:, :, 3].mul(Xa[:, :, 2] + Aa[:, :, 2].mul(Xa[:, :, 1])))
        elif Xa.size(2) == 2:
            Xa[:, :, 1].add_(Aa[:, :, 1].mul(Xa[:, :, 0]))
            return
        else:
            return

        # down sweep (first 2 steps unfolded)
        Aa = A[:, :, 2**(num_steps-2)-1:L:2**(num_steps-2)]
        Xa = X[:, :, 2**(num_steps-2)-1:L:2**(num_steps-2)]
        Xa[:, :, 2].add_(Aa[:, :, 2].mul(Xa[:, :, 1]))
        Aa[:, :, 2].mul_(Aa[:, :, 1])

        for k in range(num_steps-3, -1, -1):
            Aa = A[:, :, 2**k-1:L:2**k]
            Xa = X[:, :, 2**k-1:L:2**k]

            T = Xa.size(2)
            Aa = Aa.view(B, D, T//2, 2, -1)
            Xa = Xa.view(B, D, T//2, 2, -1)

            Xa[:, :, 1:, 0].add_(Aa[:, :, 1:, 0].mul(Xa[:, :, :-1, 1]))
            Aa[:, :, 1:, 0].mul_(Aa[:, :, :-1, 1])

    @staticmethod
    def pscan_rev(A, X):
        # A : (B, D, L, N)
        # X : (B, D, L, N)

        # the same function as above, but in reverse
        # (if you flip the input, call pscan, then flip the output, you get what this function outputs)
        # it is used in the backward pass

        # only supports L that is a power of two (mainly for a clearer code)

        B, D, L, _ = A.size()
        num_steps = int(math.log2(L))

        # up sweep (last 2 steps unfolded)
        Aa = A
        Xa = X
        for _ in range(num_steps-2):
            T = Xa.size(2)
            Aa = Aa.view(B, D, T//2, 2, -1)
            Xa = Xa.view(B, D, T//2, 2, -1)
                    
            Xa[:, :, :, 0].add_(Aa[:, :, :, 0].mul(Xa[:, :, :, 1]))
            Aa[:, :, :, 0].mul_(Aa[:, :, :, 1])

            Aa = Aa[:, :, :, 0]
            Xa = Xa[:, :, :, 0]

        # we have only 4, 2 or 1 nodes left
        if Xa.size(2) == 4:
            Xa[:, :, 2].add_(Aa[:, :, 2].mul(Xa[:, :, 3]))
            Aa[:, :, 2].mul_(Aa[:, :, 3])

            Xa[:, :, 0].add_(Aa[:, :, 0].mul(Xa[:, :, 1].add(Aa[:, :, 1].mul(Xa[:, :, 2]))))
        elif Xa.size(2) == 2:
            Xa[:, :, 0].add_(Aa[:, :, 0].mul(Xa[:, :, 1]))
            return
        else:
            return

        # down sweep (first 2 steps unfolded)
        Aa = A[:, :, 0:L:2**(num_steps-2)]
        Xa = X[:, :, 0:L:2**(num_steps-2)]
        Xa[:, :, 1].add_(Aa[:, :, 1].mul(Xa[:, :, 2]))
        Aa[:, :, 1].mul_(Aa[:, :, 2])

        for k in range(num_steps-3, -1, -1):
            Aa = A[:, :, 0:L:2**k]
            Xa = X[:, :, 0:L:2**k]

            T = Xa.size(2)
            Aa = Aa.view(B, D, T//2, 2, -1)
            Xa = Xa.view(B, D, T//2, 2, -1)

            Xa[:, :, :-1, 1].add_(Aa[:, :, :-1, 1].mul(Xa[:, :, 1:, 0]))
            Aa[:, :, :-1, 1].mul_(Aa[:, :, 1:, 0])

    @staticmethod
    def forward(ctx, A_in, X_in):
        """
        Applies the parallel scan operation, as defined above. Returns a new tensor.
        If you can, privilege sequence lengths that are powers of two.

        Args:
            A_in : (B, L, D, N)
            X_in : (B, L, D, N)

        Returns:
            H : (B, L, D, N)
        """

        L = X_in.size(1)

        # cloning is requiered because of the in-place ops
        if L == npo2(L):
            A = A_in.clone()
            X = X_in.clone()
        else:
            # pad tensors (and clone btw)
            A = pad_npo2(A_in) # (B, npo2(L), D, N)
            X = pad_npo2(X_in) # (B, npo2(L), D, N)
        
        # prepare tensors
        A = A.transpose(2, 1) # (B, D, npo2(L), N)
        X = X.transpose(2, 1) # (B, D, npo2(L), N)

        # parallel scan (modifies X in-place)
        PScan.pscan(A, X)

        ctx.save_for_backward(A_in, X)
        
        # slice [:, :L] (cut if there was padding)
        return X.transpose(2, 1)[:, :L]
    
    @staticmethod
    def backward(ctx, grad_output_in):
        """
        Flows the gradient from the output to the input. Returns two new tensors.

        Args:
            ctx : A_in : (B, L, D, N), X : (B, D, L, N)
            grad_output_in : (B, L, D, N)

        Returns:
            gradA : (B, L, D, N), gradX : (B, L, D, N)
        """

        A_in, X = ctx.saved_tensors

        L = grad_output_in.size(1)

        # cloning is requiered because of the in-place ops
        if L == npo2(L):
            grad_output = grad_output_in.clone()
            # the next padding will clone A_in
        else:
            grad_output = pad_npo2(grad_output_in) # (B, npo2(L), D, N)
            A_in = pad_npo2(A_in) # (B, npo2(L), D, N)

        # prepare tensors
        grad_output = grad_output.transpose(2, 1)
        A_in = A_in.transpose(2, 1) # (B, D, npo2(L), N)
        A = torch.nn.functional.pad(A_in[:, :, 1:], (0, 0, 0, 1)) # (B, D, npo2(L), N) shift 1 to the left (see hand derivation)

        # reverse parallel scan (modifies grad_output in-place)
        PScan.pscan_rev(A, grad_output)

        Q = torch.zeros_like(X)
        Q[:, :, 1:].add_(X[:, :, :-1] * grad_output[:, :, 1:])

        return Q.transpose(2, 1)[:, :L], grad_output.transpose(2, 1)[:, :L]
    
pscan = PScan.apply



"""

This file closely follows the mamba_simple.py from the official Mamba implementation, and the mamba-minimal by @johnma2006.
The major differences are :
-the convolution is done with torch.nn.Conv1d
-the selective scan is done in PyTorch

A sequential version of the selective scan is also available for comparison.

- A Mamba model is composed of several layers, which are ResidualBlock.
- A ResidualBlock is composed of a MambaBlock, a normalization, and a residual connection : ResidualBlock(x) = mamba(norm(x)) + x
- This leaves us with the MambaBlock : its input x is (B, L, D) and its outputs y is also (B, L, D) (B=batch size, L=seq len, D=model dim).
First, we expand x into (B, L, 2*ED) (where E is usually 2) and split it into x and z, each (B, L, ED).
Then, we apply the short 1d conv to x, followed by an activation function (silu), then the SSM.
We then multiply it by silu(z).
See Figure 3 of the paper (page 8) for a visual representation of a MambaBlock.

"""

@dataclass
class MambaConfig:
    d_model: int # D
    n_layers: int
    dt_rank: Union[int, str] = 'auto'
    d_state: int = 16 # N in paper/comments
    expand_factor: int = 2 # E in paper/comments
    d_conv: int = 4

    dt_min: float = 0.001
    dt_max: float = 0.1
    dt_init: str = "random" # "random" or "constant"
    dt_scale: float = 1.0
    dt_init_floor = 1e-4

    bias: bool = False
    conv_bias: bool = True

    pscan: bool = True # use parallel scan mode or sequential mode when training

    def __post_init__(self):
        self.d_inner = self.expand_factor * self.d_model # E*D = ED in comments

        if self.dt_rank == 'auto':
            self.dt_rank = math.ceil(self.d_model / 16)

class Mamba(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()

        self.config = config

        self.layers = nn.ModuleList([ResidualBlock(config) for _ in range(config.n_layers)])
        self.norm_f = RMSNorm(config.d_model)

    def forward(self, x):
        # x : (B, L, D)

        # y : (B, L, D)

        for layer in self.layers:
            x = layer(x)

        x = self.norm_f(x)

        return x
    
    def step(self, x, caches):
        # x : (B, L, D)
        # caches : [cache(layer) for all layers], cache : (h, inputs)

        # y : (B, L, D)
        # caches : [cache(layer) for all layers], cache : (h, inputs)

        for i, layer in enumerate(self.layers):
            x, caches[i] = layer.step(x, caches[i])

        return x, caches

class ResidualBlock(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()

        self.mixer = MambaBlock(config)
        self.norm = RMSNorm(config.d_model)

    def forward(self, x):
        # x : (B, L, D)

        # output : (B, L, D)

        output = self.mixer(self.norm(x)) + x
        return output
    
    def step(self, x, cache):
        # x : (B, D)
        # cache : (h, inputs)
                # h : (B, ED, N)
                # inputs: (B, ED, d_conv-1)

        # output : (B, D)
        # cache : (h, inputs)

        output, cache = self.mixer.step(self.norm(x), cache)
        output = output + x
        return output, cache

class MambaBlock(nn.Module):
    def __init__(self, config: MambaConfig):
        super().__init__()

        self.config = config

        # projects block input from D to 2*ED (two branches)
        self.in_proj = nn.Linear(config.d_model, 2 * config.d_inner, bias=config.bias)

        self.conv1d = nn.Conv1d(in_channels=config.d_inner, out_channels=config.d_inner, 
                              kernel_size=config.d_conv, bias=config.conv_bias, 
                              groups=config.d_inner,
                              padding=config.d_conv - 1)
        
        # projects x to input-dependent Δ, B, C
        self.x_proj = nn.Linear(config.d_inner, config.dt_rank + 2 * config.d_state, bias=False)

        # projects Δ from dt_rank to d_inner
        self.dt_proj = nn.Linear(config.dt_rank, config.d_inner, bias=True)

        # dt initialization
        # dt weights
        dt_init_std = config.dt_rank**-0.5 * config.dt_scale
        if config.dt_init == "constant":
            nn.init.constant_(self.dt_proj.weight, dt_init_std)
        elif config.dt_init == "random":
            nn.init.uniform_(self.dt_proj.weight, -dt_init_std, dt_init_std)
        else:
            raise NotImplementedError
        
        # dt bias
        dt = torch.exp(
            torch.rand(config.d_inner) * (math.log(config.dt_max) - math.log(config.dt_min)) + math.log(config.dt_min)
        ).clamp(min=config.dt_init_floor)
        inv_dt = dt + torch.log(-torch.expm1(-dt)) # inverse of softplus: https://github.com/pytorch/pytorch/issues/72759
        with torch.no_grad():
            self.dt_proj.bias.copy_(inv_dt)
        #self.dt_proj.bias._no_reinit = True # initialization would set all Linear.bias to zero, need to mark this one as _no_reinit
        # todo : explain why removed

        # S4D real initialization
        A = torch.arange(1, config.d_state + 1, dtype=torch.float32).repeat(config.d_inner, 1)
        self.A_log = nn.Parameter(torch.log(A)) # why store A in log ? to keep A < 0 (cf -torch.exp(...)) ? for gradient stability ?
        self.D = nn.Parameter(torch.ones(config.d_inner))

        # projects block output from ED back to D
        self.out_proj = nn.Linear(config.d_inner, config.d_model, bias=config.bias)

    def forward(self, x):
        # x : (B, L, D)
        
        # y : (B, L, D)

        _, L, _ = x.shape

        xz = self.in_proj(x) # (B, L, 2*ED)
        x, z = xz.chunk(2, dim=-1) # (B, L, ED), (B, L, ED)

        # x branch
        x = x.transpose(1, 2) # (B, ED, L)
        x = self.conv1d(x)[:, :, :L] # depthwise convolution over time, with a short filter
        x = x.transpose(1, 2) # (B, L, ED)

        x = F.silu(x)
        y = self.ssm(x)

        # z branch
        z = F.silu(z)

        output = y * z
        output = self.out_proj(output) # (B, L, D)

        return output
    
    def ssm(self, x):
        # x : (B, L, ED)

        # y : (B, L, ED)

        A = -torch.exp(self.A_log.float()) # (ED, N)
        D = self.D.float()
        # TODO remove .float()

        deltaBC = self.x_proj(x) # (B, L, dt_rank+2*N)

        delta, B, C = torch.split(deltaBC, [self.config.dt_rank, self.config.d_state, self.config.d_state], dim=-1) # (B, L, dt_rank), (B, L, N), (B, L, N)
        delta = F.softplus(self.dt_proj(delta)) # (B, L, ED)

        if self.config.pscan:
            y = self.selective_scan(x, delta, A, B, C, D)
        else:
            y = self.selective_scan_seq(x, delta, A, B, C, D)

        return y
    
    def selective_scan(self, x, delta, A, B, C, D):
        # x : (B, L, ED)
        # Δ : (B, L, ED)
        # A : (ED, N)
        # B : (B, L, N)
        # C : (B, L, N)
        # D : (ED)

        # y : (B, L, ED)

        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, L, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(2) # (B, L, ED, N)

        BX = deltaB * (x.unsqueeze(-1)) # (B, L, ED, N)
        
        hs = pscan(deltaA, BX)

        y = (hs @ C.unsqueeze(-1)).squeeze(3) # (B, L, ED, N) @ (B, L, N, 1) -> (B, L, ED, 1)

        y = y + D * x

        return y
    
    def selective_scan_seq(self, x, delta, A, B, C, D):
        # x : (B, L, ED)
        # Δ : (B, L, ED)
        # A : (ED, N)
        # B : (B, L, N)
        # C : (B, L, N)
        # D : (ED)

        # y : (B, L, ED)

        _, L, _ = x.shape

        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, L, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(2) # (B, L, ED, N)

        BX = deltaB * (x.unsqueeze(-1)) # (B, L, ED, N)

        h = torch.zeros(x.size(0), self.config.d_inner, self.config.d_state, device=deltaA.device) # (B, ED, N)
        hs = []

        for t in range(0, L):
            h = deltaA[:, t] * h + BX[:, t]
            hs.append(h)
            
        hs = torch.stack(hs, dim=1) # (B, L, ED, N)

        y = (hs @ C.unsqueeze(-1)).squeeze(3) # (B, L, ED, N) @ (B, L, N, 1) -> (B, L, ED, 1)

        y = y + D * x

        return y
    
    # -------------------------- inference -------------------------- #
    """
    Concerning auto-regressive inference

    The cool part of using Mamba : inference is constant wrt to sequence length
    We just have to keep in cache, for each layer, two things :
    - the hidden state h (which is (B, ED, N)), as you typically would when doing inference with a RNN
    - the last d_conv-1 inputs of the layer, to be able to compute the 1D conv which is a convolution over the time dimension
      (d_conv is fixed so this doesn't incur a growing cache as we progress on generating the sequence)
      (and d_conv is usually very small, like 4, so we just have to "remember" the last 3 inputs)

    Concretely, these two quantities are put inside a cache tuple, and are named h and inputs respectively.
    h is (B, ED, N), and inputs is (B, ED, d_conv-1)
    The MambaBlock.step() receives this cache, and, along with outputing the output, alos outputs the updated cache for the next call.

    The cache object is initialized as follows : (None, torch.zeros()).
    When h is None, the selective scan function detects it and start with h=0.
    The torch.zeros() isn't a problem (it's same as just feeding the input, because the conv1d is padded)

    As we need one such cache variable per layer, we store a caches object, which is simply a list of cache object. (See mamba_lm.py)
    """
    
    def step(self, x, cache):
        # x : (B, D)
        # cache : (h, inputs)
                # h : (B, ED, N)
                # inputs : (B, ED, d_conv-1)
        
        # y : (B, D)
        # cache : (h, inputs)
        
        h, inputs = cache
        
        xz = self.in_proj(x) # (B, 2*ED)
        x, z = xz.chunk(2, dim=1) # (B, ED), (B, ED)

        # x branch
        x_cache = x.unsqueeze(2)
        x = self.conv1d(torch.cat([inputs, x_cache], dim=2))[:, :, self.config.d_conv-1] # (B, ED)

        x = F.silu(x)
        y, h = self.ssm_step(x, h)

        # z branch
        z = F.silu(z)

        output = y * z
        output = self.out_proj(output) # (B, D)

        # prepare cache for next call
        inputs = torch.cat([inputs[:, :, 1:], x_cache], dim=2) # (B, ED, d_conv-1)
        cache = (h, inputs)
        
        return output, cache

    def ssm_step(self, x, h):
        # x : (B, ED)
        # h : (B, ED, N)

        # y : (B, ED)
        # h : (B, ED, N)

        A = -torch.exp(self.A_log.float()) # (ED, N) # todo : ne pas le faire tout le temps, puisque c'est indépendant de la timestep
        D = self.D.float()
        # TODO remove .float()

        deltaBC = self.x_proj(x) # (B, dt_rank+2*N)

        delta, B, C = torch.split(deltaBC, [self.config.dt_rank, self.config.d_state, self.config.d_state], dim=-1) # (B, dt_rank), (B, N), (B, N)
        delta = F.softplus(self.dt_proj(delta)) # (B, ED)

        deltaA = torch.exp(delta.unsqueeze(-1) * A) # (B, ED, N)
        deltaB = delta.unsqueeze(-1) * B.unsqueeze(1) # (B, ED, N)

        BX = deltaB * (x.unsqueeze(-1)) # (B, ED, N)

        if h is None:
            h = torch.zeros(x.size(0), self.config.d_inner, self.config.d_state, device=deltaA.device) # (B, ED, N)

        h = deltaA * h + BX # (B, ED, N)

        y = (h @ C.unsqueeze(-1)).squeeze(2) # (B, ED, N) @ (B, N, 1) -> (B, ED, 1)

        y = y + D * x

        # todo : pq h.squeeze(1) ??
        return y, h.squeeze(1)

# taken straight from https://github.com/johnma2006/mamba-minimal/blob/master/model.py
class RMSNorm(nn.Module):
    def __init__(self, d_model: int, eps: float = 1e-5):
        super().__init__()

        self.eps = eps
        self.weight = nn.Parameter(torch.ones(d_model))

    def forward(self, x):
        output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps) * self.weight

        return output

import math

import torch
import torch.nn.functional as F

"""

An implementation of the parallel scan operation in PyTorch (Blelloch version).
Please see docs/pscan.ipynb for a detailed explanation of what happens here.

"""

def npo2(len):
    """
    Returns the next power of 2 above len
    """

    return 2 ** math.ceil(math.log2(len))

def pad_npo2(X):
    """
    Pads input length dim to the next power of 2

    Args:
        X : (B, L, D, N)

    Returns:
        Y : (B, npo2(L), D, N)
    """

    len_npo2 = npo2(X.size(1))
    pad_tuple = (0, 0, 0, 0, 0, len_npo2 - X.size(1))
    return F.pad(X, pad_tuple, "constant", 0)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
# from mamba import Mamba, MambaConfig
import argparse
seed = 1
epochs = 100
lr = 0.0001
wd = 1e-5
hidden = 16
layer = 2
n_test = 300
ts_code = 601988
use_cuda = False
cuda = use_cuda and torch.cuda.is_available()
# parser = argparse.ArgumentParser()
# parser.add_argument('--use-cuda', default=False,
#                     help='CUDA training.')
# parser.add_argument('--seed', type=int, default=1, help='Random seed.')
# parser.add_argument('--epochs', type=int, default=100,
#                     help='Number of epochs to train.')
# parser.add_argument('--lr', type=float, default=0.01,
#                     help='Learning rate.')
# parser.add_argument('--wd', type=float, default=1e-5,
#                     help='Weight decay (L2 loss on parameters).')
# parser.add_argument('--hidden', type=int, default=16,
#                     help='Dimension of representations')
# parser.add_argument('--layer', type=int, default=2,
#                     help='Num of layers')
# parser.add_argument('--n-test', type=int, default=300,
#                     help='Size of test set')
# parser.add_argument('--ts-code', type=str, default='601988',
#                     help='Stock code')                    

# args = parser.parse_args()
# cuda = use_cuda and torch.cuda.is_available()

def evaluation_metric(y_test,y_hat):
    MSE = mean_squared_error(y_test, y_hat)
    RMSE = MSE**0.5
    MAE = mean_absolute_error(y_test,y_hat)
    R2 = r2_score(y_test,y_hat)
    

    print('%.4f %.4f %.4f %.4f' % (MSE,RMSE,MAE,R2))

def set_seed(seed,cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)

def dateinf(series, n_test):
    lt = len(series)
    print('Training start',series[0])
    print('Training end',series[lt-n_test-1])
    print('Testing start',series[lt-n_test])
    print('Testing end',series[lt-1])

set_seed(seed,cuda)

class Net(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.config = MambaConfig(d_model=hidden, n_layers=layer)
        self.mamba = nn.Sequential(
            nn.Linear(in_dim,hidden),
            Mamba(self.config),
            nn.Linear(hidden,out_dim),
            nn.Tanh()
            
        )
    
    def forward(self,x):
        x = self.mamba(x)
        return x.flatten()

def PredictWithData(trainX, trainy, testX):
    clf = Net(len(trainX[0]),1)
    opt = torch.optim.Adam(clf.parameters(),lr=lr,weight_decay=wd)
    xt = torch.from_numpy(trainX).float().unsqueeze(0)
    xv = torch.from_numpy(testX).float().unsqueeze(0)
    yt = torch.from_numpy(trainy).float()
    if cuda:
        clf = clf.cuda()
        xt = xt.cuda()
        xv = xv.cuda()
        yt = yt.cuda()
    
    for e in range(epochs):
        clf.train()
        z = clf(xt)
        loss = F.mse_loss(z,yt)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if e%10 == 0 and e!=0:
            print('Epoch %d | Lossp: %.4f' % (e, loss.item()))

    clf.eval()
    mat = clf(xv)
    if cuda: mat = mat.cpu()
    yhat = mat.detach().numpy().flatten()
    return yhat


In [3]:
#import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")


symbol_to_fetch = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-05-01'
def fetch_ticker_data(symbol, start_date, end_date):
    """Fetches stock data for a given symbol using yfinance."""
    ticker = yf.Ticker(symbol)
    data = ticker.history(start='1980-01-01', end=end_date)
    return data

def label_data(data):
    # Calculate the percentage change in price from one day to the next
    data['Percentage Change'] = data['Close'].pct_change()
    data['Percentage Change'] = data['Percentage Change'].shift(-1)
    # data['Sentiment'] = pd.Series(np.where(data['Percentage Change'] > 0.025, 1, np.where(data['Percentage Change'] < -0.025, -1, 0)), index=data.index)
    # data['perc_change'] = data['Percentage Change']
    # # Drop any rows with missing values
    # data.dropna(inplace=True)
    # data.drop('Percentage Change',axis=1 , inplace=True)
    return data

#fetching data 
stock = fetch_ticker_data(symbol_to_fetch, start_date, end_date)
# Calculate deltas for open, high, low, and close columns
for i in range(1, 30):  # Calculate deltas up to 5 days
    stock[f"open_delta_{i}day"] = stock["Open"].diff(periods=i)
    stock[f"high_delta_{i}day"] = stock["High"].diff(periods=i)
    stock[f"low_delta_{i}day"] = stock["Low"].diff(periods=i)
    stock[f"close_delta_{i}day"] = stock["Close"].diff(periods=i)
        # Rolling mean and standard deviation of OHLC prices
    stock['Rolling_Mean_Open_{i}day'] = stock['Open'].rolling(window=i).mean()
    stock['Rolling_Mean_High_{i}day'] = stock['High'].rolling(window=i).mean()
    stock['Rolling_Mean_Low_{i}day'] = stock['Low'].rolling(window=i).mean()
    stock['Rolling_Mean_Close_{i}day'] = stock['Close'].rolling(window=i).mean()

    stock['Rolling_Std_Open_{i}day'] = stock['Open'].rolling(window=i).std()
    stock['Rolling_Std_High_{i}day'] = stock['High'].rolling(window=i).std()
    stock['Rolling_Std_Low_{i}day'] = stock['Low'].rolling(window=i).std()
    stock['Rolling_Std_Close_{i}day'] = stock['Close'].rolling(window=i).std()
stock = stock.fillna(method="ffill", axis=0)
stock = stock.fillna(method="bfill", axis=0)
# stock.index = stock.index.date

# Split the data into training and test sets
train_data_index = np.searchsorted(stock.index.values, np.datetime64(start_date))
train_data = stock.iloc[:train_data_index]
test_data = stock.loc[start_date:]
train_data = label_data(train_data)
test_data = label_data(test_data)
train_data.fillna(method='ffill',axis = 0, inplace=True)
test_data.fillna(method='ffill',axis = 0, inplace=True)
#trian & test data
X_train_data = train_data.iloc[:,:-1]
y_train_data = train_data.iloc[:,-1]
X_test_data = test_data.iloc[:,:-1]
y_test_data = test_data.iloc[:,-1]
print(len(X_test_data))
# Normalize the data
normalizer = MinMaxScaler()
X_train_data_normalizer = normalizer.fit_transform(X_train_data)
X_test_data_normalizer = normalizer.transform(X_test_data)

# # # Reshape X_train_data_normalizer
# X_train = X_train_data_normalizer.reshape(X_train_data_normalizer.shape[0], X_train_data_normalizer.shape[1], 1)
# X_test = X_test_data_normalizer.reshape(X_test_data_normalizer.shape[0], X_test_data_normalizer.shape[1], 1)


1089


In [4]:
y_train_data.values

array([-0.05217053, -0.07339807,  0.024751  , ...,  0.0059353 ,
        0.00730624,  0.00730624])

In [5]:
trainy

NameError: name 'trainy' is not defined

In [6]:
# predictions = PredictWithData(trainX, trainy, testX)
predictions = PredictWithData(X_train_data_normalizer, y_train_data.values, X_test_data_normalizer)

Epoch 10 | Lossp: 0.0199
Epoch 20 | Lossp: 0.0033
Epoch 30 | Lossp: 0.0009
Epoch 40 | Lossp: 0.0008
Epoch 50 | Lossp: 0.0008
Epoch 60 | Lossp: 0.0008
Epoch 70 | Lossp: 0.0008
Epoch 80 | Lossp: 0.0008
Epoch 90 | Lossp: 0.0008


In [7]:
test_data['Sentiment'] = pd.Series(np.where(test_data['Percentage Change'] > 0, 1, -1), index=test_data.index)


In [8]:
test_data['transformer_sentiment'] = predictions
test_data['transformer_sentiment'] = test_data['transformer_sentiment'].apply(lambda x: -1 if x< 0 else 1)



In [9]:
test_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,open_delta_1day,high_delta_1day,low_delta_1day,...,high_delta_28day,low_delta_28day,close_delta_28day,open_delta_29day,high_delta_29day,low_delta_29day,close_delta_29day,Percentage Change,Sentiment,transformer_sentiment
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00-05:00,71.962075,73.021202,71.707013,72.960472,135480400,0.0,0.0,1.532829,1.681013,1.377365,...,8.385544,8.451129,9.026844,6.884296,7.919130,7.238955,8.273788,-0.009722,-1,-1
2020-01-03 00:00:00-05:00,72.183128,73.016335,72.025232,72.251144,146322800,0.0,0.0,0.221053,-0.004867,0.318219,...,8.883519,8.579878,8.604164,7.678640,8.380677,8.769348,8.317516,0.007968,1,1
2020-01-06 00:00:00-05:00,71.366911,72.865711,71.114274,72.826843,118387200,0.0,0.0,-0.816217,-0.150624,-0.910959,...,8.934521,7.751511,9.235737,7.311830,8.732895,7.668920,9.179863,-0.004703,-1,1
2020-01-07 00:00:00-05:00,72.836571,73.094064,72.263288,72.484344,108872000,0.0,0.0,1.469661,0.228353,1.149014,...,8.370950,8.492416,7.778236,9.048702,9.162874,8.900525,8.893238,0.016086,1,1
2020-01-08 00:00:00-05:00,72.185556,73.954000,72.185556,73.650352,132079200,0.0,0.0,-0.651015,0.859936,-0.077732,...,9.055998,8.419553,9.449524,8.368530,9.230886,8.414685,8.944244,0.021241,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24 00:00:00-04:00,166.314410,169.070681,165.984870,168.791061,48251800,0.0,0.0,1.188378,2.246955,1.288264,...,-5.003213,-5.832091,-3.974609,-6.221577,-3.884735,-4.543829,-2.107147,0.005147,1,-1
2024-04-25 00:00:00-04:00,169.300369,170.378908,167.922233,169.659882,50558300,0.0,0.0,2.985959,1.308227,1.937363,...,-2.007277,-2.137106,-2.726303,-3.375428,-3.694986,-3.894728,-3.105789,-0.003473,-1,-1
2024-04-26 00:00:00-04:00,169.649895,171.107909,168.950831,169.070679,44838400,0.0,0.0,0.349526,0.729001,1.028598,...,-6.361388,-4.334138,-4.414017,-1.288257,-1.278276,-1.108508,-3.315506,0.024808,1,-1


In [10]:

evaluation_metric(test_data['Percentage Change'], predictions)

0.0016 0.0404 0.0218 -2.8157


In [12]:
from sklearn.metrics import classification_report
print(classification_report(test_data['Sentiment'],test_data['transformer_sentiment']))

from sklearn.metrics import accuracy_score,recall_score,precision_score


# compute accuracy
accuracy = accuracy_score(test_data['Sentiment'],test_data['transformer_sentiment'])
precision = precision_score(test_data['Sentiment'],test_data['transformer_sentiment'])
recall = recall_score(test_data['Sentiment'],test_data['transformer_sentiment'])
print(f"Accuracy: {accuracy:.2f} , Precision : {precision:.2f} , Recall : {recall:.2f}")


              precision    recall  f1-score   support

          -1       0.48      0.64      0.55       527
           1       0.52      0.36      0.42       562

    accuracy                           0.50      1089
   macro avg       0.50      0.50      0.49      1089
weighted avg       0.50      0.50      0.49      1089

Accuracy: 0.50 , Precision : 0.52 , Recall : 0.36


# Mamba Architecture + Technical Indicators + Feature Engineering + Sequencing of Data

In [43]:
#import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import datetime
import warnings
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

warnings.filterwarnings("ignore")


symbol_to_fetch = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-05-01'
# Parameters
batch_size = 256
sequence_length = 30
stride = 1

def fetch_ticker_data(symbol, start_date, end_date):
    """Fetches stock data for a given symbol using yfinance."""
    ticker = yf.Ticker(symbol)
    data = ticker.history(start='1980-01-01', end=end_date)
    return data

def label_data(data):
    # Calculate the percentage change in price from one day to the next
    data['pr_change_on_last_day'] = data['Close'].pct_change()
    data['pr_change_on_current_day'] = data['pr_change_on_last_day'].shift(-1)
    data.iloc[0,-2] = 0
    # data['sentiment'] = pd.Series(np.where(data['pr_change_on_current_day'] > 0, 1, np.where(data['pr_change_on_current_day'] < 0, -1, 0)), index=data.index)
    # data['perc_change'] = data['Percentage Change']
    # # Drop any rows with missing values
    # data.dropna(inplace=True)
    # data.drop('pr_change_on_current_day',axis=1 , inplace=True)
    return data
stock = fetch_ticker_data(symbol_to_fetch, start_date, end_date)

# Calculate deltas, moving averages, and Bollinger Bands
for i in range(1, 90,5):
    stock[f"open_delta_{i}day"] = stock["Open"].diff(periods=i)
    stock[f"high_delta_{i}day"] = stock["High"].diff(periods=i)
    stock[f"low_delta_{i}day"] = stock["Low"].diff(periods=i)
    stock[f"close_delta_{i}day"] = stock["Close"].diff(periods=i)
    stock[f"rolling_mean_open_{i}day"] = stock["Open"].rolling(window=i).mean()
    stock[f"rolling_mean_high_{i}day"] = stock["High"].rolling(window=i).mean()
    stock[f"rolling_mean_low_{i}day"] = stock["Low"].rolling(window=i).mean()
    stock[f"rolling_mean_close_{i}day"] = stock["Close"].rolling(window=i).mean()
    stock[f"rolling_std_open_{i}day"] = stock["Open"].rolling(window=i).std()
    stock[f"rolling_std_high_{i}day"] = stock["High"].rolling(window=i).std()
    stock[f"rolling_std_low_{i}day"] = stock["Low"].rolling(window=i).std()
    stock[f"rolling_std_close_{i}day"] = stock["Close"].rolling(window=i).std()

stock['fast_ma'] = stock['Close'].rolling(window=20).mean()
stock['slow_ma'] = stock['Close'].rolling(window=50).mean()
stock['bollinger_high'] = stock['Close'].rolling(window=20).mean() + (2 * stock['Close'].rolling(window=20).std())
stock['bollinger_low'] = stock['Close'].rolling(window=20).mean() - (2 * stock['Close'].rolling(window=20).std())
stock['ema'] = stock['Close'].ewm(span=20, adjust=False).mean()
stock['envelope_high'] = stock['Close'].rolling(window=20).mean() * (1 + 0.05)
stock['envelope_low'] = stock['Close'].rolling(window=20).mean() * (1 - 0.05)
stock['macd_line'] = stock['Close'].ewm(span=12, adjust=False).mean() - stock['Close'].ewm(span=26, adjust=False).mean()
stock['macd_signal'] = stock['macd_line'].ewm(span=9, adjust=False).mean()

# RSI calculation
def calculate_rsi(data, rsi_period):
    delta = data['Close'].diff().dropna()
    gain = delta.where(delta > 0, 0).dropna()
    loss = -delta.where(delta < 0, 0).dropna()
    avg_gain = gain.rolling(window=rsi_period).mean()
    avg_loss = loss.rolling(window=rsi_period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

stock['rsi'] = calculate_rsi(stock, 14)

# # Stochastic Oscillator calculation
# def calculate_stochastic(data, k_window, d_window):
#     high_low = data[['High', 'Low']]
#     c = data['Close']
#     highest = high_low.rolling(window=k_window).max()
#     lowest = high_low.rolling(window=k_window).min()
#     print(((c - lowest) / (highest - lowest)) * 100)
#     stochastic_k = ((c - lowest) / (highest - lowest)) * 100
#     stochastic_d = stochastic_k.rolling(window=d_window).mean()
#     return stochastic_k, stochastic_d
# stock['stochastic_k'], stock['stochastic_d'] = calculate_stochastic(stock, 14, 3)

# stock['stochastic_k']= calculate_stochastic(stock, 14, 3)[0]
# stock['stochastic_d']= calculate_stochastic(stock, 14, 3)[1]
stock['day'] = pd.to_datetime(stock.index).day
stock['month'] = pd.to_datetime(stock.index).month
stock['year'] = pd.to_datetime(stock.index).year
stock['weekday'] = pd.to_datetime(stock.index).weekday
stock['dayofyear'] = pd.to_datetime(stock.index).dayofyear
stock = stock.fillna(method="ffill", axis=0)
stock = stock.fillna(method="bfill", axis=0)
stock.index = stock.index.date
# Split the data into training and test sets

# df = stock.copy()

# # Calculate pairwise correlation
# corr_matrix = df.corr()

# # Identify highly correlated columns
# redundant_cols = set()
# for i in range(5,len(corr_matrix.columns)-1):
#     for j in range(i+1, len(corr_matrix.columns)):
#         if corr_matrix.iloc[i,j] > 0.8 and corr_matrix.columns[i] not in redundant_cols:
#             redundant_cols.add(corr_matrix.columns[j])

# # Remove one of the redundant columns
# for col in redundant_cols:
#     df = df.drop(col, axis=1)

# # Print the updated DataFrame
# print(df)

# stock = df.copy()
train_data_index = np.searchsorted(stock.index.values, np.datetime64(start_date))
train_data = stock.iloc[:int(0.9*train_data_index)].copy()
val_data  = stock.iloc[int(0.9*train_data_index)-sequence_length:train_data_index].copy()
test_data = stock.iloc[train_data_index-sequence_length:].copy()
train_data = label_data(train_data)
val_data = label_data(val_data)
test_data = label_data(test_data)
train_data.fillna(0,axis = 0, inplace=True)
val_data.fillna(0,axis = 0, inplace=True)
test_data.fillna(0,axis = 0, inplace=True)

#trian & test data
X_train_data = train_data.iloc[:,:-1]
y_train_data = train_data.iloc[:,-1]
#trian & test data
X_val_data = val_data.iloc[:,:-1]
y_val_data = val_data.iloc[:,-1]
X_test_data = test_data.iloc[:,:-1]
y_test_data = test_data.iloc[:,-1]
print(len(X_test_data), len(X_test_data.columns))
from keras.utils import to_categorical

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train_data, num_classes=3)
y_val_onehot = to_categorical(y_val_data, num_classes=3)

y_train_data = to_categorical(y_train_data)
y_test_data = to_categorical(y_test_data)
y_val_data = to_categorical(y_val_data)

# Normalize the data
normalizer = MinMaxScaler()
X_train_data_normalizer = normalizer.fit_transform(X_train_data)
X_val_data_normalizer = normalizer.fit_transform(X_val_data)
X_test_data_normalizer = normalizer.transform(X_test_data)

# # # Reshape X_train_data_normalizer
X_train_reshaped = X_train_data_normalizer.reshape(X_train_data_normalizer.shape[0], X_train_data_normalizer.shape[1], 1)
X_val_reshaped = X_val_data_normalizer.reshape(X_val_data_normalizer.shape[0], X_val_data_normalizer.shape[1], 1)
X_test_reshaped = X_test_data_normalizer.reshape(X_test_data_normalizer.shape[0], X_test_data_normalizer.shape[1], 1)

def create_sequences(x,y,sequence_length,stride):
    sequence_length  = sequence_length
    X_test_data_normalizer_sequences = []
    y_test_data_sequences = []
    stride = stride
    no_of_rows = len(x)
    no_of_columns = len(x[0])
    for i in range(sequence_length, no_of_rows-1 , stride):
        X_test_data_normalizer_sequences.append(x[i-sequence_length: i])
        y_test_data_sequences.append(y[i-1])
    return np.array(X_test_data_normalizer_sequences),np.array(y_test_data_sequences)
        
X_train_data_normalizer_sequences,y_train_data_sequences = create_sequences(X_train_data_normalizer,y_train_data,sequence_length,stride)
X_test_data_normalizer_sequences,y_test_data_sequences = create_sequences(X_test_data_normalizer,y_test_data,sequence_length,stride)
train_dataset = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X_train_data_normalizer,
    y_train_data,
    length = 3,
    sampling_rate=1,
    stride=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    reverse=False,
    batch_size=batch_size
)
val_dataset = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X_val_data_normalizer,
    y_val_data,
    length = sequence_length,
    sampling_rate=1,
    stride=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    reverse=False,
    batch_size=batch_size
)
test_dataset = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    X_test_data_normalizer,
    y_test_data,
    length = sequence_length,
    sampling_rate=1,
    stride=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    reverse=False,
    batch_size=batch_size
)

# predictions = PredictWithData(trainX, trainy, testX)
predictions = PredictWithData(X_train_data_normalizer, y_train_data, X_test_data_normalizer)
test_data['Sentiment'] = pd.Series(np.where(test_data['pr_change_on_current_day'] > 0, 1, -1), index=test_data.index)
test_data['transformer_sentiment'] = predictions
test_data['transformer_sentiment'] = test_data['transformer_sentiment'].apply(lambda x: -1 if x< 0 else 1)
print(evaluation_metric(test_data['pr_change_on_current_day'], predictions))
from sklearn.metrics import classification_report
print(classification_report(test_data['Sentiment'],test_data['transformer_sentiment']))

from sklearn.metrics import accuracy_score,recall_score,precision_score


# compute accuracy
accuracy = accuracy_score(test_data['Sentiment'],test_data['transformer_sentiment'])
precision = precision_score(test_data['Sentiment'],test_data['transformer_sentiment'])
recall = recall_score(test_data['Sentiment'],test_data['transformer_sentiment'])
print(f"Accuracy: {accuracy:.2f} , Precision : {precision:.2f} , Recall : {recall:.2f}")

1119 239
Epoch 10 | Lossp: 0.1654
Epoch 20 | Lossp: 0.0518
Epoch 30 | Lossp: 0.0274
Epoch 40 | Lossp: 0.0188
Epoch 50 | Lossp: 0.0148
Epoch 60 | Lossp: 0.0123
Epoch 70 | Lossp: 0.0106
Epoch 80 | Lossp: 0.0093
Epoch 90 | Lossp: 0.0083
0.7508 0.8665 0.8657 -1792.7887
None
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       539
           1       0.52      1.00      0.68       580

    accuracy                           0.52      1119
   macro avg       0.26      0.50      0.34      1119
weighted avg       0.27      0.52      0.35      1119

Accuracy: 0.52 , Precision : 0.52 , Recall : 1.00


1.0